# Preprocesamiento de datos

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tabulate import tabulate
import os

## Lectura y preparación inicial de los datos

### Cargar el dataset

In [2]:
file_path = "../data/raw/data.csv"
df = pd.read_csv(file_path, header=None)

### Preparar los datos

In [3]:
# Crear nombres cortos para las características
feature_names = [f'feat{str(i+1).zfill(2)}' for i in range(30)]

# Eliminar columna ID y asignar nombres a las columnas
df = df.drop([0], axis=1)  # El ID no aporta información
df.columns = ['diagnosis'] + feature_names

# Mapear diagnóstico de M/B a 0/1
df['diagnosis'] = df['diagnosis'].map({'M': 0, 'B': 1})

## Dividir en características (X) y target (y)

In [4]:
X = df[feature_names]
y = df['diagnosis']

## Dividir en conjuntos de entrenamiento y test
Proponemos estos porcentajes para la división: 80% - 20%
Pero los dejamos como parámetros.

In [5]:
# Parámetro del porcentaje destinado a entrenamiento / test
train_size = 0.8
test_size = 1 - train_size

X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42,
    stratify=y  # Mantener la proporción de clases en ambos conjuntos
)

In [ ]:
# 5. Normalización Z-score usando SOLO los datos de entrenamiento
# Calcular media y desviación estándar del conjunto de entrenamiento
mean_train = X_train.mean()
std_train = X_train.std()

# Normalizar conjuntos de entrenamiento y test
X_train_normalized = (X_train - mean_train) / std_train
X_test_normalized = (X_test - mean_train) / std_train

# 6. Guardar los parámetros de normalización
normalization_params = {
    'mean': mean_train,
    'std': std_train
}

# 7. Crear directorio para datos procesados si no existe
output_dir = "../data/processed"
os.makedirs(output_dir, exist_ok=True)

# 8. Guardar los conjuntos normalizados y los parámetros
# Conjuntos de entrenamiento
train_df = pd.DataFrame(X_train_normalized, columns=feature_names)
train_df.insert(0, 'diagnosis', y_train.values)
train_df.to_csv(f"{output_dir}/train_normalized.csv", index=False)

# Conjuntos de test
test_df = pd.DataFrame(X_test_normalized, columns=feature_names)
test_df.insert(0, 'diagnosis', y_test.values)
test_df.to_csv(f"{output_dir}/test_normalized.csv", index=False)

# Parámetros de normalización
params_df = pd.DataFrame({
    'mean': mean_train,
    'std': std_train
})
params_df.to_csv(f"{output_dir}/normalization_params.csv")

# 9. Imprimir información sobre los conjuntos de datos
print("\nInformación sobre la división de datos:")
print(f"Tamaño total del dataset: {len(df)}")
print(f"Tamaño del conjunto de entrenamiento: {len(X_train)}")
print(f"Tamaño del conjunto de test: {len(X_test)}")

# Distribución de clases
print("\nDistribución de clases:")
print("Conjunto de entrenamiento:")
print(y_train.value_counts(normalize=True).round(3))
print("\nConjunto de test:")
print(y_test.value_counts(normalize=True).round(3))

# Mostrar primeras filas del conjunto de entrenamiento normalizado
print("\nPrimeras filas del conjunto de entrenamiento normalizado:")
print(tabulate(train_df.head(), headers='keys', tablefmt='rst', showindex=True))

# Verificar que los parámetros de normalización se guardaron correctamente
print("\nParámetros de normalización guardados en:", f"{output_dir}/normalization_params.csv")